# Assignment 2: Scale-Space Blob Detection

### The goal of this assignment is to implement a Laplacian blob detector

Noel Mrowiec
10/1/2024
mrowiec3

In [94]:
# Libraries you will find useful
import numpy as np
import scipy 
import skimage
import matplotlib.pyplot as plt
import os
from PIL import Image, ImageOps 
# for applying the Laplacian filter
from scipy.ndimage.filters import gaussian_laplace, maximum_filter

/tmp/ipykernel_964/681115042.py:9: DeprecationWarning: Please import `gaussian_laplace` from the `scipy.ndimage` namespace; the `scipy.ndimage.filters` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.filters import gaussian_laplace, maximum_filter
/tmp/ipykernel_964/681115042.py:9: DeprecationWarning: Please import `maximum_filter` from the `scipy.ndimage` namespace; the `scipy.ndimage.filters` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.filters import gaussian_laplace, maximum_filter


In [95]:
# Function to globally normalize a 2D array to the range [0, 1] 
def normalize(data):
    return (data - data.min()) / (data.max() - data.min())

In [96]:

# load each image 
dir_path = "images/"
images = []     #color images
filenames = []
for filename in os.listdir(dir_path):
    if filename.endswith('.jpg'):
        filenames.append(filename)
        img_path = os.path.join(dir_path, filename)
        img = Image.open(img_path)
        #img.show()  # This will display the image
        
        # convert images to grayscale
        img_gs = ImageOps.grayscale(img) 
        #img_gs.show()
        img_gs = np.array(img_gs).astype(float)
        img_gs = normalize(img_gs)
        images.append(img_gs)
        

print(f"Loaded {len(images)} images")

Loaded 4 images


In [97]:
def get_sigma(k):
    return (2 ** k)

In [98]:

def laplacian_gaussian(image, sigmas):
    scale_space = np.empty((image.shape[0],image.shape[1], len(sigmas)))

    for k, sigma in enumerate(sigmas):
        #sigma = get_sigma(k)
        lap_gaus = gaussian_laplace(image, sigma, mode='nearest') 
        scale_norm_LoG = (sigma ** 2) * lap_gaus                                                      #scale normalized
        scale_space[:,:,k] = scale_norm_LoG

    return scale_space

In [99]:
import numpy as np
from scipy.ndimage import gaussian_laplace, maximum_filter

# Example image
image = np.random.rand(10, 10)
sigmas = range(1, 5)  # Sigma values from 1 to 11

print(image)

# Create a 3D scale space
scale_space = np.stack([gaussian_laplace(image, sigma=sigma) for sigma in sigmas], axis=-1)

# Non-maximum suppression in 3D
footprint = np.ones((3, 3, 3))
local_maxima = (scale_space == maximum_filter(scale_space, footprint=footprint))

# Get coordinates of the local maxima
coordinates = np.argwhere(local_maxima)

# Print the coordinates and corresponding sigma
for coord in coordinates:
    x, y, z = coord
    sigma = sigmas[z]
    print(f"Local maxima at (x={x}, y={y}) with sigma={sigma}")



[[0.04205565 0.42776679 0.56595632 0.31852087 0.92940258 0.18493746
  0.92983502 0.90103086 0.59460215 0.75254709]
 [0.87734154 0.04624858 0.91152614 0.99467679 0.55734927 0.11425965
  0.13795283 0.67232501 0.65211937 0.75517112]
 [0.89579245 0.71980572 0.73656211 0.34110974 0.49870361 0.72597515
  0.77051917 0.95527356 0.02156914 0.78546131]
 [0.23875205 0.92992455 0.40625891 0.17066837 0.16286893 0.25841787
  0.81131896 0.37637698 0.05735544 0.0821428 ]
 [0.3738478  0.6090499  0.38061084 0.28249347 0.64249146 0.19310181
  0.15308721 0.91650369 0.01304225 0.68325669]
 [0.07953389 0.17995678 0.05902825 0.06079156 0.08597171 0.13602029
  0.98385023 0.71601213 0.45250037 0.13977104]
 [0.07880639 0.62469504 0.08928744 0.29050631 0.10054161 0.74637624
  0.13027052 0.90967    0.98660908 0.1709443 ]
 [0.13862294 0.24782965 0.54885786 0.77804914 0.30935198 0.52629295
  0.5086438  0.06837547 0.06722556 0.34380682]
 [0.36909885 0.7274915  0.86605801 0.38004774 0.47791713 0.69441415
  0.69760136

In [100]:
from scipy.ndimage.filters import rank_filter

def non_max_suppression(array, threshold):
    local_max = rank_filter(array, rank=-1, size=13)

    # Only keep the pixels that are local maxima
    result = np.where(array == local_max, array, 0)

    #suppress any result less than threshold
    result = np.where(result < threshold, 0, array)
    return result

/tmp/ipykernel_964/1999998974.py:1: DeprecationWarning: Please import `rank_filter` from the `scipy.ndimage` namespace; the `scipy.ndimage.filters` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.filters import rank_filter


In [101]:
def threshold_scale_space(lap_of_gauss, threshold):
    #suppress any result less than threshold
    result = np.where(lap_of_gauss < threshold, 0, lap_of_gauss)
    return result

In [102]:
#TODO remove
from skimage import feature
def get_local_peak_coord(stacked_images):

    lm = feature.peak_local_max(stacked_images, threshold_abs=10, footprint=np.ones((3, 3, 3)), threshold_rel=10, exclude_border=True)
    return lm[0], lm[1]

In [103]:
# def get_xy_coordinates(image):
#     # Non-maximum suppression in 3D
#     footprint = np.ones((3, 3, 3))
#     local_maxima = (scale_space == maximum_filter(scale_space, footprint=footprint))

#     # Get coordinates of the local maxima
#     coordinates = np.argwhere(local_maxima)

#     # Print the coordinates and corresponding sigma
#     for coord in coordinates:
#         x, y, z = coord
#         sigma = sigmas[z]
#         print(f"Local maxima at (x={x}, y={y}) with sigma={sigma}"


In [104]:
def non_max_suppression_3d(scale_space, sigmas):

    # Non-maximum suppression in 3D
    footprint = np.ones((3, 3, len(sigmas)))
    local_maxima = (scale_space == maximum_filter(scale_space, footprint=footprint))

    return local_maxima

In [ ]:
def get_maxima_coords(scale_space, threshold):
    """
    returns: a list of coordinates of local maxima as a tuple with (x,y,sigma)
    """
    # Get coordinates of the local maxima
    coordinates = np.argwhere(local_maxima)

    # Filter coordinates by the threshold
    filtered_coords = [coord for coord in coordinates if scale_space[tuple(coord)] > threshold] #source: Copilot


    # # Print the coordinates and corresponding sigma
    # for coord in filtered_coords:
    #     x, y, z = coord
    #     sigma = sigmas[z]
    #     print(f"Local maxima at (x={x}, y={y}) with sigma={sigma}")

    return filtered_coords

In [105]:
# constant
sqt_2 = np.sqrt(2)

In [106]:
#N = 10
SIGMAS = [1,2,4,8,16,32]
coordinates_per_image = []

mean_percent = 1

for image in images:
    lap_of_gauss = laplacian_gaussian(image, SIGMAS)
    #take absolute value
    lap_of_gauss = np.abs(lap_of_gauss)

    #threshold scale space
    threshold = np.percentile(lap_of_gauss, 90)
    # lap_of_gauss = threshold_scale_space(lap_of_gauss, threshold)

    #3D non-maximum suppression
    local_maxima = non_max_suppression_3d(lap_of_gauss, SIGMAS)
    coordinates = get_local_peak_coord(local_maxima, threshold)
    coordinates_per_image.append(coordinates)

    # for k, sigma in enumerate(SIGMAS):
       
    #     #non_max_suppressed = non_max_suppression(lap_of_gauss[:,:, k], threshold)

    #     # Convert to Pandas DataFrame
    #     # df = pd.DataFrame(non_max_suppressed)

    #     # print(df.describe())


    #     #h_pos, w_pos = get_xy_coordinates(non_max_suppressed)
    #     h_pos, w_pos = get_local_peak_coord()

    #     x_coordinates = np.append(x_coordinates, w_pos)
    #     y_coordinates = np.append(y_coordinates, h_pos)

    #     radii = np.append(radii, np.full(h_pos.shape[0], sigma * sqt_2))       # TODO divide by sigma squared?




Local maxima at (x=0, y=30) with sigma=2
Local maxima at (x=0, y=51) with sigma=4
Local maxima at (x=0, y=92) with sigma=32
Local maxima at (x=0, y=111) with sigma=1
Local maxima at (x=0, y=149) with sigma=2
Local maxima at (x=0, y=165) with sigma=2
Local maxima at (x=0, y=173) with sigma=1
Local maxima at (x=0, y=175) with sigma=1
Local maxima at (x=0, y=182) with sigma=16
Local maxima at (x=0, y=191) with sigma=1
Local maxima at (x=0, y=205) with sigma=8
Local maxima at (x=0, y=245) with sigma=1
Local maxima at (x=0, y=259) with sigma=2
Local maxima at (x=0, y=273) with sigma=8
Local maxima at (x=0, y=292) with sigma=4
Local maxima at (x=0, y=307) with sigma=1
Local maxima at (x=0, y=336) with sigma=8
Local maxima at (x=0, y=370) with sigma=16
Local maxima at (x=0, y=397) with sigma=4
Local maxima at (x=0, y=408) with sigma=4
Local maxima at (x=0, y=418) with sigma=4
Local maxima at (x=0, y=420) with sigma=8
Local maxima at (x=0, y=428) with sigma=1
Local maxima at (x=0, y=449) with 

In [107]:





# filtering the image (two implementations)
# one that increases filter size, and one that downsamples the image
# For timing, use time.time()



# nonmaximum suppression in scale space
# you may find functions scipy.ndimage.filters.rank_filter or scipy.ndimage.filters.generic_filter useful



# To display the detected regions as circle
from matplotlib.patches import Circle
def show_all_circles(image, cx, cy, rad, color='r'):
    """
    image: numpy array, representing the grayscsale image
    cx, cy: numpy arrays or lists, centers of the detected blobs
    rad: numpy array or list, radius of the detected blobs
    """
    fig, ax = plt.subplots()
    ax.set_aspect('equal')
    ax.imshow(image, cmap='gray')
    for x, y, r in zip(cx, cy, rad):
        circ = Circle((x, y), r, color=color, fill=False)
        ax.add_patch(circ)

    plt.title('%i circles' % len(cy))

    plt.show()

In [108]:
#np.set_printoptions(threshold=np.inf)

# print(x_coordinates)
# print(y_coordinates)
print(radii)

print(x_coordinates.max())
print(y_coordinates.max())

print(f"image size {images[0].shape}")
print(x_coordinates.shape, y_coordinates.shape, radii.shape)
show_all_circles(images[0], x_coordinates, y_coordinates, radii)
#show_all_circles(images[0], [55,33,88,10], [76,350,12,100], [5,7,1,2])

[]


ValueError: zero-size array to reduction operation maximum which has no identity

In [96]:
import cv2
import numpy as np

# Load your image
image = cv2.imread('images/butterfly.jpg', cv2.IMREAD_GRAYSCALE)

# Initialize SIFT detector
sift = cv2.SIFT_create()

# Detect keypoints and compute descriptors
keypoints, descriptors = sift.detectAndCompute(image, None)

# Print the radius (scale) of each keypoint
for kp in keypoints:
    print(f"Keypoint radius: {kp.size}")

print(f"number of keypoints {len(keypoints)}")


Keypoint radius: 7.039892673492432
Keypoint radius: 7.039892673492432
Keypoint radius: 9.89643383026123
Keypoint radius: 9.89643383026123
Keypoint radius: 9.936521530151367
Keypoint radius: 8.38396167755127
Keypoint radius: 11.6674165725708
Keypoint radius: 11.6674165725708
Keypoint radius: 10.20260238647461
Keypoint radius: 10.20260238647461
Keypoint radius: 5.025646686553955
Keypoint radius: 5.977290153503418
Keypoint radius: 8.488493919372559
Keypoint radius: 5.123463153839111
Keypoint radius: 5.123463153839111
Keypoint radius: 8.046290397644043
Keypoint radius: 9.387319564819336
Keypoint radius: 9.387319564819336
Keypoint radius: 7.613601207733154
Keypoint radius: 17.341140747070312
Keypoint radius: 14.746243476867676
Keypoint radius: 9.3104887008667
Keypoint radius: 14.199297904968262
Keypoint radius: 6.085587024688721
Keypoint radius: 6.085587024688721
Keypoint radius: 7.732848644256592
Keypoint radius: 9.687704086303711
Keypoint radius: 5.872124195098877
Keypoint radius: 9.58001